<a href="https://colab.research.google.com/github/elliespathtostatistics/data-science-prep/blob/master/oscartakehomecondensed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from collections import defaultdict
import pickle
import seaborn as sns

root_dir = '/content/drive/My Drive/oscar_interview/'

prescription_drugs = pd.read_csv(root_dir + 'prescription_drugs.csv')
claim_lines = pd.read_csv(root_dir + 'claim_lines.csv')
ccs = pd.read_csv(root_dir + 'ccs.csv')


#### The code follows a 3 step process: exploratory data analysis (EDA), data preprocessing, then model building. The three datasets are prescription drugs, claim lines, and clinical classification system (CCS) codes. ####

**Section I. EDA: Data Loading and Initial Exploration:**

*  I calculate the number of unique categories in each column of the `ccs` DataFrame to determine an appropriate prediction response variable that's not too sparse to build a model on
*  In ccs, ccs_1_des is the least granular classification, followed by ccs_2_des, and ccs_3_des being the most granular classification
* prescriptions have the most number of unique of rows ~3mln rows, and claims have ~ 2mln rows
* In claim lines, 24 rows have empty dates of service
* In claim lines, a handful of rows have dates of service in year 1899 which seem to be errors in recording especially since problem prompt states that all files contain medical utilization simulations for 200,000 members over a period of 3 years
* 2013-12-04 is the earliest date of service in prescriptions so we can use that to splice rows off in claims assoicated w an earlier date of service
* By looking at a single member, for example, M0291613, in claims and in prescriptions, one sees that prescriptions usually take place after claims rather than on the same date so joining svc_date can not be used to join the two files



In [ ]:
prescription_drugs.head()
claim_lines.head()
ccs.head()

prescription_drugs.dtypes
claim_lines.dtypes
ccs.dtypes

prescription_drugs.shape
claim_lines.shape
ccs.shape

prescription_drugs.isnull().sum()
claim_lines.isnull().sum()
ccs.isnull().sum()

,0
diag,0
diag_desc,0
ccs_1_desc,0
ccs_2_desc,0
ccs_3_desc,0


In [ ]:
# Calculate the number of unique categories for each column
category_counts = ccs.nunique()

# May want to use the column with the fewest categories as response variable forprediction purposes

In [ ]:

# Count unique values for specified columns
unique_drug_categories = prescription_drugs['drug_category'].nunique()
unique_drug_groups = prescription_drugs['drug_group'].nunique()
unique_drug_classes = prescription_drugs['drug_class'].nunique()

print(f"Number of unique drug categories: {unique_drug_categories}")
print(f"Number of unique drug groups: {unique_drug_groups}")
print(f"Number of unique drug classes: {unique_drug_classes}")

Number of unique drug categories: 92
Number of unique drug groups: 464
Number of unique drug classes: 687


#### **Section II. Data Cleanup** #####

1. **Data Type Conversion:** The `date_svc` column in both `prescription_drugs` and `claim_lines` needs to be converted from string objects to date objects.

2. **Data Cleaning:** The `diag1` column in `claim_lines` contains periods (`.`) that need to be removed for a correct join with the `diag` column in the css dataset.

3. **Join Issue:** Joining `prescriptions` and `claims` on `member` and `date_svc` is problematic due to different `date_svc` values. Instead I use `member_id` for an exact match, then select the `date_svc` from the claim immediately preceding the prescription for that member.

5. **Strip out NAs that dont provide any signal:** Joining the combined `claims` and `diagnosis` data with `prescriptions` will result in missing values in `claim_lines` columns in the merged file.  Rows with these missing values are removed from the final `drugs_merged_claim_diag` dataset.




In [ ]:
prescription_drugs['date_svc'] = pd.to_datetime(prescription_drugs['date_svc'])
claim_lines['date_svc'] = pd.to_datetime(claim_lines['date_svc'])
claim_lines[(claim_lines['member_id']== 'M0291613')].sort_values(by='date_svc')
prescription_drugs[(prescription_drugs['member_id']== 'M0291613')].sort_values(by='date_svc')
claim_lines['diag1'] = claim_lines['diag1'].str.replace('.', '')
merged_claims_diag = pd.merge(claim_lines, ccs, left_on = 'diag1', right_on = 'diag')

merged_claims_diag.dropna(inplace = True)


merged_drugs_claims_diag = pd.merge_asof(prescription_drugs.sort_values('date_svc'), merged_claims_diag.sort_values('date_svc'), by = 'member_id', on = 'date_svc', direction = 'backward')

merged_drugs_claims_diag.shape
merged_drugs_claims_diag.isnull().sum()

# get rid of rows in merged_drugs_claims_diag where diag is null

merged_drugs_claims_diag_nona = merged_drugs_claims_diag.dropna(subset=['diag'], inplace=False)
merged_drugs_claims_diag_nona.count()

# store for later use in case notebook shuts down
merged_drugs_claims_diag_nona.to_csv(root_dir + 'merged_drugs_claims_diag_nona.csv')

In [ ]:
merged_drugs_claims_diag_nona

,record_id_x,member_id,date_svc,ndc,drug_category,drug_group,drug_class,record_id_y,diag1,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
29084,1241525768392412139,M0274305,2014-06-03,378081005,Diuretics,Thiazides and Thiazide-Like Diuretics,Thiazides and Thiazide-Like Diuretics,49.0,S83249A,S83249A,"Oth tear of medial meniscus, current injury, u...",Injury and poisoning,Joint disorders and dislocations; trauma-relat...,Joint disorders and dislocations; trauma-related
40043,8429749976728463572,M0274305,2014-07-06,378081005,Diuretics,Thiazides and Thiazide-Like Diuretics,Thiazides and Thiazide-Like Diuretics,49.0,S83249A,S83249A,"Oth tear of medial meniscus, current injury, u...",Injury and poisoning,Joint disorders and dislocations; trauma-relat...,Joint disorders and dislocations; trauma-related
47169,1901169414130553768,M0274305,2014-07-26,74662490,Thyroid Agents,Thyroid Hormones,Thyroid Hormones,49.0,S83249A,S83249A,"Oth tear of medial meniscus, current injury, u...",Injury and poisoning,Joint disorders and dislocations; trauma-relat...,Joint disorders and dislocations; trauma-related
53039,728025933617431401,M0061560,2014-08-11,603459315,Corticosteroids,Glucocorticosteroids,Glucocorticosteroids,51.0,M25569,M25569,Pain in unspecified knee,Diseases of the musculoskeletal system and con...,Non-traumatic joint disorders,Other non-traumatic joint disorders
53055,1138697338770219249,M0061560,2014-08-11,603307821,Musculoskeletal Therapy Agents,Central Muscle Relaxants,Central Muscle Relaxants,51.0,M25569,M25569,Pain in unspecified knee,Diseases of the musculoskeletal system and con...,Non-traumatic joint disorders,Other non-traumatic joint disorders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005929,9210244504415370965,M0049547,2018-06-04,378427693,Antivirals,Herpes Agents,Herpes Agents - Purine Analogues,1754856.0,B009,B009,"Herpesviral infection, unspecified",Infectious and parasitic diseases,Viral infection,Viral infection
3005930,5884260829561633062,M0127579,2018-06-04,591521550,Anti-Infective Agents - Misc.,Anti-Infective Agents - Misc.,Anti-Infective Agents - Misc.,1866389.0,Z7189,Z7189,Other specified counseling,Symptoms; signs; and ill-defined conditions an...,Factors influencing health care,Administrative/social admission
3005931,3791176895834354679,M0053558,2018-06-04,93719856,Antidepressants,Selective Serotonin Reuptake Inhibitors (SSRIs),Selective Serotonin Reuptake Inhibitors (SSRIs),1866084.0,Z0001,Z0001,Encounter for general adult medical exam w abn...,Symptoms; signs; and ill-defined conditions an...,Factors influencing health care,Medical examination/evaluation
3005932,8547122146919190365,M0142858,2018-06-04,51672403001,Anticoagulants,Coumarin Anticoagulants,Coumarin Anticoagulants,1742832.0,Z86711,Z86711,Personal history of pulmonary embolism,Diseases of the circulatory system,Diseases of the heart,Pulmonary heart disease


**Section III. Data prep and model in order to characterize a member’s health status based on their outpatient data**




In [ ]:
## Create the generated dataframes and dictionaries needed to determine health status based on prescription drugs

# create a member to ndc map so that eventually we can provide a member and we can build a characterization of a member’s health status based on their outpatient data
member_to_ndc = merged_drugs_claims_diag_nona.groupby('member_id')['ndc'].apply(list).to_dict()

# Count the number of different diagnoses mapped to each ndc
diag_counts_by_ndc = pd.DataFrame()
diag_counts_by_ndc['count_unique_diag'] = merged_drugs_claims_diag_nona.groupby('ndc')['diag'].transform('nunique')
merged_drugs_claims_diag_nona.groupby(['ndc']).nunique('diag').sort_values(by='diag', ascending=False).head(30)
print(f"The number of unique NDC codes is: {merged_drugs_claims_diag_nona['ndc'].nunique()}")

# each ndc maybe given to multiple diagnoses, we may want to predict ccs descriptions rather than diagnosis since the diagnosis column may be too sparse for some diagnoses

# Group by NDC and diagnosis description, count occurrences, and sort
diagnosis_counts = merged_drugs_claims_diag_nona.groupby(['ndc', 'diag_desc', 'diag']).size().reset_index(name='counts')

# Count the number of unique diagnoses for each NDC
diag_counts = merged_drugs_claims_diag_nona.groupby('ndc')['diag'].nunique()

# Get the top diagnoses for each NDC
top_5_diagnoses = diagnosis_counts.groupby('ndc').head(5)

# get unique ccs_1_desc per ndc
unique_ccs_counts = merged_drugs_claims_diag_nona.groupby('ndc')['ccs_1_desc'].nunique().reset_index()


The number of unique NDC codes is: 18354


In [ ]:
top_5_diagnoses

,ndc,diag_desc,diag,counts
0,2143301,"Acute maxillary sinusitis, unspecified",J0100,3
1,2143301,"Chronic kidney disease, stage 3 (moderate)",N183,1
2,2143301,Coronary angioplasty status,Z9861,1
3,2143380,Abnormal electrocardiogram [ECG] [EKG],R9431,3
4,2143380,Abnormal findings on diagnostic imaging of bod...,R938,1
...,...,...,...,...
829661,99207085060,Acne vulgaris,L700,1
829662,99207085060,Diseases of lips,K130,1
829663,99207085060,Dyshidrosis [pompholyx],L301,1
829664,99207085060,Encntr screen mammogram for malignant neoplasm...,Z1231,1


In [ ]:
merged_drugs_claims_diag_nona.groupby(['ndc']).nunique('diag').sort_values(by='diag', ascending=False).head(30)


,record_id_x,member_id,date_svc,drug_category,drug_group,drug_class,record_id_y,diag1,diag,diag_desc,ccs_1_desc,ccs_2_desc,ccs_3_desc
ndc,,,,,,,,,,,,,
59310057922,18118,11301,991,1,1,1,15952,1806,1806,1804,18,120,205
54327099,16738,9191,993,1,1,1,14320,1796,1796,1793,17,110,194
93310905,6371,5376,937,1,1,1,6023,1424,1424,1421,17,107,190
68180012202,4017,3485,934,1,1,1,3916,1381,1381,1379,18,103,180
93015010,3732,2675,860,1,1,1,3468,1339,1339,1336,18,109,200
116200116,4483,3340,941,1,1,1,4032,1318,1318,1316,18,109,192
65162062711,5076,2384,884,1,1,1,4302,1310,1310,1309,18,114,201
64380073706,7418,3591,939,1,1,1,6143,1259,1259,1257,17,105,188
59746000103,6645,5615,948,1,1,1,6521,1225,1225,1225,18,102,171


In [ ]:
# save master dataframe merged_drugs_claims_diag_nona to dict for use later

drug_diag_list = merged_drugs_claims_diag_nona.to_dict('records')
drug_to_diag_des = defaultdict(list)

# construct a ndc to diag dictionary
for row in drug_diag_list:
  if row['ndc'] in drug_to_diag_des:
    drug_to_diag_des[row['ndc']].append((row['diag'], row['diag_desc']))
  else:
    drug_to_diag_des[row['ndc']] = [(row['diag'], row['diag_desc'])]

# save the ndc_to_diag mapping to a file
with open("drug_to_diag_desc.pkl", "wb") as f:
    pickle.dump(drug_to_diag_des, f)

with open("drug_to_diag_desc.pkl", "rb") as f:
    drug_to_diag_des = pickle.load(f)

# create function to take in a list of ndc prescriptions and output statuses to get the diagnoses for a list of ndcs
def gen_health_status(ndc_list):
  health_conditions = []
  for drug in ndc_list:
    if drug in drug_to_diag_des:
      health_conditions.append(drug_to_diag_des[drug])

  return health_conditions




In [ ]:
# using ndc to diag screened in too many diagnoses so limiting the diagnoses to the top 5 diagnnoses per ndc and store in dict
top_diag_by_ndc_dict = top_5_diagnoses.to_dict('records')

# use the top 5 diagnosis by ndc to limit diagnosis and increase accuracy
drug_to_top_diag = defaultdict(list)

for row in top_diag_by_ndc_dict:
  if row['ndc'] in drug_to_top_diag:
    drug_to_top_diag[row['ndc']].append((row['diag'], row['diag_desc']))
  else:
    drug_to_top_diag[row['ndc']] = [row['diag'], row['diag_desc']]

with open("drug_to_top_diag.pkl", "wb") as f:
    pickle.dump(drug_to_top_diag, f)

with open("drug_to_top_diag.pkl", "rb") as f:
    drug_to_top_diag  = pickle.load(f)

In [ ]:
drug_to_top_diag

defaultdict(list,
            {2143301: ['J0100',
              'Acute maxillary sinusitis, unspecified',
              ('N183', 'Chronic kidney disease, stage 3 (moderate)'),
              ('Z9861', 'Coronary angioplasty status')],
             2143380: ['R9431',
              'Abnormal electrocardiogram [ECG] [EKG]',
              ('R938',
               'Abnormal findings on diagnostic imaging of body structures'),
              ('R930',
               'Abnormal findings on dx imaging of skull and head, NEC'),
              ('R748', 'Abnormal levels of other serum enzymes'),
              ('S40811A', 'Abrasion of right upper arm, initial encounter')],
             2143401: ['Z23', 'Encounter for immunization'],
             2143480: ['R799',
              'Abnormal finding of blood chemistry, unspecified',
              ('R931',
               'Abnormal findings on dx imaging of heart and cor circ'),
              ('L83', 'Acanthosis nigricans'),
              ('L570', 'Actinic kera

In [ ]:
# create a function that given a member_id, maps the ndcs for that member, and the top 5 diagnoses associated with each ndc
def gen_top_health_status_given_member(member_id):
    ndc_list = member_to_ndc[member_id]
    health_conditions = list()

    for drug in ndc_list:
        if drug in drug_to_top_diag:
            health_conditions.append(drug_to_top_diag[drug])


    unique_health_conditions = []
    seen = set()
    for condition in health_conditions:
        if condition[0] not in seen:
            unique_health_conditions.append(condition[1])
            seen.add(condition[0])

    print(f"The member {member_id} has diagnoses of:")
    for condition in unique_health_conditions:
        print(f"{condition}")
    #return unique_health_conditions

**Section V. Build model to predict statuses based solely on prescription drugs info**

In [ ]:
# function that uses top diagnoses for each ndc, takes an ndc or a list of ndcs and outputs the top diagnoses associated with each ndc
def gen_top_health_status(ndc_list):

    health_conditions = []  # Change to a list to store all conditions
    for drug in ndc_list:
        if drug in drug_to_top_diag:
            health_conditions.append(drug_to_top_diag[drug]) # append to list instead of add to set


    unique_health_conditions = []
    seen = set()
    for condition in health_conditions:
        if condition[0] not in seen:
            unique_health_conditions.append(condition[1])
            seen.add(condition[0])

    print("Top conditions associated with your ndc include the following:")
    for condition in unique_health_conditions:
        print(f"{condition}")

    return unique_health_conditions

In [ ]:
gen_top_health_status_given_member('M0078348')

The member M0078348 has diagnoses of:
Abdominal distension (gaseous)
Acute pharyngitis, unspecified
Acute nasopharyngitis [common cold]
Abnormal electrocardiogram [ECG] [EKG]
Acute bronchitis, unspecified
Abrasion, right knee, initial encounter
10 weeks gestation of pregnancy
Abnormal levels of other serum enzymes


In [ ]:
# helper function to get ndcs given member_id
def get_member_ndcs(member_id):
  return member_to_ndc[member_id]



In [ ]:
example_ndcs = get_member_ndcs('M0000001')
example_statuses = gen_top_health_status(example_ndcs)

Top conditions associated with your ndc include the following:
Abdominal distension (gaseous)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming 'merged_drugs_claims_diag_nona' is already defined and loaded

# Prepare the data
X = merged_drugs_claims_diag_nona[['drug_category', 'drug_class', 'drug_group']]
y = merged_drugs_claims_diag_nona['ccs_1_desc']

# Encode categorical features
le_drug_category = LabelEncoder()
le_drug_class = LabelEncoder()
le_drug_group = LabelEncoder()
le_ccs = LabelEncoder()

X.loc[:, 'drug_category'] = le_drug_category.fit_transform(X['drug_category'])
X.loc[:, 'drug_class'] = le_drug_class.fit_transform(X['drug_class'])
X.loc[:, 'drug_group'] = le_drug_group.fit_transform(X['drug_group'])
y = le_ccs.fit_transform(y)


with open("encoders.pickle","wb") as f:
    pickle.dump(le_drug_category, f)
    pickle.dump(le_drug_class, f)
    pickle.dump(le_drug_group, f)
    pickle.dump(le_ccs, f)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the decision tree classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

with open("decision_tree_model.pickle","wb") as f:
    pickle.dump(clf, f)

# Make predictions
y_pred = clf.predict(X_test)


# Evaluate the model (example using accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.3095183813682936


In [ ]:
import pickle
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def predict_diagnosis(ndc):
    try:
      with open('decision_tree_model.pickle', 'rb') as file:
        clf = pickle.load(file)
    except FileNotFoundError:
      print("Model file not found. Please train the model and save it first.")
      return

    try:
      with open("encoders.pickle", "rb") as f:
        le_drug_category = pickle.load(f)
        le_drug_class = pickle.load(f)
        le_drug_group = pickle.load(f)
        le_ccs = pickle.load(f)
    except FileNotFoundError:
      print("Encoder file not found. Please train encoders and save it first.")
      return

    prescription_drugs = pd.read_csv('/content/drive/My Drive/oscar_interview/prescription_drugs.csv')
    try:
      drug_info = prescription_drugs[prescription_drugs['ndc'] == ndc].iloc[0]
      drug_category = drug_info['drug_category']
      drug_class = drug_info['drug_class']
      drug_group = drug_info['drug_group']
    except IndexError:
      print(f"NDC {ndc} not found in the prescription_drugs table.")
      return None  # Or handle the error appropriately
    # Encode the drug information using the loaded LabelEncoders
    try:
      encoded_drug_category = le_drug_category.transform([drug_category])[0]
      encoded_drug_class = le_drug_class.transform([drug_class])[0]
      encoded_drug_group = le_drug_group.transform([drug_group])[0]

      X_rows = [[encoded_drug_category, encoded_drug_class, encoded_drug_group]]
      X_new = pd.DataFrame(X_rows, columns=['drug_category', 'drug_class', 'drug_group'])

      # Make a prediction using the loaded model
      prediction = clf.predict(X_new)

      # Decode the prediction using the loaded LabelEncoder
      predicted_diagnosis = le_ccs.inverse_transform(prediction)[0]

      # Print or return the prediction
      print('')
      print(f"The provided NDC {ndc} belongs to\n\tdrug category: '{drug_category}',\n\tdrug class: '{drug_class}', \n\tdrug group: '{drug_group}'.")
      print('')
      print(f"Based on this prescription, I predict the member has the following conditions: \n '{predicted_diagnosis}'")
    except ValueError as e:
      print(f"Error during prediction: {e}. Check if input values are in the range of values seen during training.")
      return None  # Or handle error in a different way

# Get user input for NDC
user_ndc = input("Please enter drug information by specifying the NDC: ")

# Try to convert to int, if fails to convert, handle gracefully
try:
    user_ndc = int(user_ndc)
    predict_diagnosis(user_ndc)
except ValueError:
    print("Invalid input. Please enter a valid integer NDC.")

Please enter drug information by specifying the NDC: 93078201

The provided NDC 93078201 belongs to
	drug category: 'Antineoplastics and Adjunctive Therapies',
	drug class: 'Antiestrogens', 
	drug group: 'Antineoplastic - Hormonal and Related Agents'.

Based on this prescription, I predict the member has the following conditions: 
 'Neoplasms'


In [ ]:
ex_drug_to_top_diag = gen_top_health_status([93078201])

Top conditions associated with your ndc include the following:
Acquired absence of bilateral breasts and nipples


In [ ]:
predict_diagnosis(99207085060)


The provided NDC 99207085060 belongs to
	drug category: 'Dermatologicals',
	drug class: 'Imidazole-Related Antifungals - Topical', 
	drug group: 'Antifungals - Topical'.

Based on this prescription, I predict the member has the following conditions: 
 'Diseases of the skin and subcutaneous tissue'


In [ ]:
ex_drug_to_top_diag = gen_top_health_status([99207085060])

Top conditions associated with your ndc include the following:
Acne vulgaris


In [ ]:
predict_diagnosis(93053601)


The provided NDC 93053601 belongs to
	drug category: 'Analgesics - Anti-Inflammatory',
	drug class: 'Nonsteroidal Anti-Inflammatory Agents (NSAIDs)', 
	drug group: 'Nonsteroidal Anti-Inflammatory Agents (NSAIDs)'.

Based on this prescription, I predict the member has the following conditions: 
 'Diseases of the musculoskeletal system and connective tissue'


In [ ]:
ex_drug_to_top_diag = gen_top_health_status([93053601])

Top conditions associated with your ndc include the following:
Strain of muscle, fascia and tendon of lower back, init


If you'd like to have more specific information about the patient's status, see the following

---



#### With more time, I would investigate multilabel models since each ndc can technically be mapped to multiple diagnosis so to say that only one diagnosis is the correct one underestimates the accuracy. Also in terms of accuracy, I would use the hamming loss to account for the multi label nature of the response variable.####

#### Additionally, I would tune tune my model parameters further to yield better accuracy/precision scores. With more time, I'd use spark to speed up the model parameter tuning.####